# Visualize

In [1]:
import fiftyone as fo
from fiftyone import ViewField as F

# dataset basics
dataset_label = 'video-game-screenshots'

## Prepare Dataset

In [2]:
if dataset_label in fo.list_datasets():
    dataset = fo.load_dataset(dataset_label)
    session = fo.launch_app(dataset, auto=False)

Session launched. Run `session.show()` to open the App in a cell output.


Open [FiftyOne 🔗](http://localhost:5151).

## Histogram

In [3]:
# Define some interesting plots
plot_1 = fo.CategoricalHistogram("years", order="frequency")
plot_2 = fo.CategoricalHistogram("platform", order="frequency")
plot_3 = fo.CategoricalHistogram("countries", order="frequency")

# Construct a custom dashboard of plots
plot = fo.ViewGrid([plot_1, plot_2, plot_3], shape=(2,2), init_view=dataset)
plot.show(height=1000)

FigureWidget({
    'data': [{'hovertemplate': '<b>years: %{x}</b><br>count: %{y}<extra></extra>',
              'marker': {'color': '#FF6D04'},
              'type': 'bar',
              'uid': '32f522e0-e0e9-4523-96f4-9bf33e0b7219',
              'x': [1988, 1989, 1986, 1990, 1985, 1984, 1983, 1987, 1982, 1981,
                    1980, 1979, 1978, 1977, 1971, 1976, 1975, 1973, 1966, 1969,
                    1968, 1972, 1962, 1974],
              'xaxis': 'x',
              'y': [18263, 16688, 15514, 13616, 12720, 10922, 10085, 10017, 6392,
                    3194, 1945, 733, 464, 191, 129, 102, 55, 50, 34, 24, 8, 3, 3,
                    2],
              'yaxis': 'y'},
             {'hovertemplate': '<b>platform: %{x}</b><br>count: %{y}<extra></extra>',
              'marker': {'color': '#FF6D04'},
              'type': 'bar',
              'uid': '5b759085-c952-49c2-aebf-3f66e3380768',
              'x': [Commodore 64, DOS, ZX Spectrum, Amiga, Atari ST, NES, Arcade,
            

## Embeddings

In [4]:
colorschemes = ['aggrnyl', 'agsunset', 'algae', 'amp', 'armyrose', 'balance',
    'blackbody', 'bluered', 'blues', 'blugrn', 'bluyl', 'brbg',
    'brwnyl', 'bugn', 'bupu', 'burg', 'burgyl', 'cividis', 'curl',
    'darkmint', 'deep', 'delta', 'dense', 'earth', 'edge', 'electric',
    'emrld', 'fall', 'geyser', 'gnbu', 'gray', 'greens', 'greys',
    'haline', 'hot', 'hsv', 'ice', 'icefire', 'inferno', 'jet',
    'magenta', 'magma', 'matter', 'mint', 'mrybm', 'mygbm', 'oranges',
    'orrd', 'oryel', 'oxy', 'peach', 'phase', 'picnic', 'pinkyl',
    'piyg', 'plasma', 'plotly3', 'portland', 'prgn', 'pubu', 'pubugn',
    'puor', 'purd', 'purp', 'purples', 'purpor', 'rainbow', 'rdbu',
    'rdgy', 'rdpu', 'rdylbu', 'rdylgn', 'redor', 'reds', 'solar',
    'spectral', 'speed', 'sunset', 'sunsetdark', 'teal', 'tealgrn',
    'tealrose', 'tempo', 'temps', 'thermal', 'tropic', 'turbid',
    'turbo', 'twilight', 'viridis', 'ylgn', 'ylgnbu', 'ylorbr',
    'ylorrd']
colorscheme = 'spectral'
    
min_dist_s = [0.001, 0.01, 0.5]
num_neighbors_s = [100, 500, 750]

for min_dist in min_dist_s:
    for num_neighbors in num_neighbors_s:

        print('Min. Distance', min_dist, 'Num. Neighbors', num_neighbors)
        
        plot = dataset.load_brain_results("dinov2_umap_"+str(min_dist).replace(".", "")+"_"+str(num_neighbors)+"_vis").visualize(
            #labels=F("kmeans_dinov2_embeddings_cluster"),
            labels=F("year"),
            colorscale=colorscheme,
            marker_size=2
        )
        plot.show(height=340, width=520)

Min. Distance 0.001 Num. Neighbors 100


FigureWidget({
    'data': [{'customdata': array(['66b5dd16818ebd7af71d6601', '66b5dd16818ebd7af71d6602',
                                   '66b5dd16818ebd7af71d6603', ..., '66b5dd3b818ebd7af71f22e9',
                                   '66b5dd3b818ebd7af71f22ea', '66b5dd3b818ebd7af71f22eb'], dtype=object),
              'hovertemplate': ('<b>label: %{marker.color}</b><' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([1985, 1989, 1982, ..., 1988, 1988, 1990]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction'},
                         'colorscale': [[0.0, 'rgb(158,1,66)'], [0.1,
                                        'rgb(213,62,79)'], [0.2,
                                        'rgb(244,109,67)'], [0.3,
                                        'rgb(253,174,97)'], [0.4,
                                        'rgb(254,224,139)'], [0.5,
                                        'rgb(255,255,191)'], [0.6,
                              

Min. Distance 0.001 Num. Neighbors 500


FigureWidget({
    'data': [{'customdata': array(['66b5dd16818ebd7af71d6601', '66b5dd16818ebd7af71d6602',
                                   '66b5dd16818ebd7af71d6603', ..., '66b5dd3b818ebd7af71f22e9',
                                   '66b5dd3b818ebd7af71f22ea', '66b5dd3b818ebd7af71f22eb'], dtype=object),
              'hovertemplate': ('<b>label: %{marker.color}</b><' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([1985, 1989, 1982, ..., 1988, 1988, 1990]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction'},
                         'colorscale': [[0.0, 'rgb(158,1,66)'], [0.1,
                                        'rgb(213,62,79)'], [0.2,
                                        'rgb(244,109,67)'], [0.3,
                                        'rgb(253,174,97)'], [0.4,
                                        'rgb(254,224,139)'], [0.5,
                                        'rgb(255,255,191)'], [0.6,
                              

Min. Distance 0.001 Num. Neighbors 750


...

FigureWidget({
    'data': [{'customdata': array(['66b5dd16818ebd7af71d6601', '66b5dd16818ebd7af71d6602',
                                   '66b5dd16818ebd7af71d6603', ..., '66b5dd3b818ebd7af71f22e9',
                                   '66b5dd3b818ebd7af71f22ea', '66b5dd3b818ebd7af71f22eb'], dtype=object),
              'hovertemplate': ('<b>label: %{marker.color}</b><' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([1985, 1989, 1982, ..., 1988, 1988, 1990]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction'},
                         'colorscale': [[0.0, 'rgb(158,1,66)'], [0.1,
                                        'rgb(213,62,79)'], [0.2,
                                        'rgb(244,109,67)'], [0.3,
                                        'rgb(253,174,97)'], [0.4,
                                        'rgb(254,224,139)'], [0.5,
                                        'rgb(255,255,191)'], [0.6,
                              

Min. Distance 0.5 Num. Neighbors 500


FigureWidget({
    'data': [{'customdata': array(['66b5dd16818ebd7af71d6601', '66b5dd16818ebd7af71d6602',
                                   '66b5dd16818ebd7af71d6603', ..., '66b5dd3b818ebd7af71f22e9',
                                   '66b5dd3b818ebd7af71f22ea', '66b5dd3b818ebd7af71f22eb'], dtype=object),
              'hovertemplate': ('<b>label: %{marker.color}</b><' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([1985, 1989, 1982, ..., 1988, 1988, 1990]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction'},
                         'colorscale': [[0.0, 'rgb(158,1,66)'], [0.1,
                                        'rgb(213,62,79)'], [0.2,
                                        'rgb(244,109,67)'], [0.3,
                                        'rgb(253,174,97)'], [0.4,
                                        'rgb(254,224,139)'], [0.5,
                                        'rgb(255,255,191)'], [0.6,
                              

Min. Distance 0.5 Num. Neighbors 750


FigureWidget({
    'data': [{'customdata': array(['66b5dd16818ebd7af71d6601', '66b5dd16818ebd7af71d6602',
                                   '66b5dd16818ebd7af71d6603', ..., '66b5dd3b818ebd7af71f22e9',
                                   '66b5dd3b818ebd7af71f22ea', '66b5dd3b818ebd7af71f22eb'], dtype=object),
              'hovertemplate': ('<b>label: %{marker.color}</b><' ... ': %{customdata}<extra></extra>'),
              'marker': {'color': array([1985, 1989, 1982, ..., 1988, 1988, 1990]),
                         'colorbar': {'len': 1, 'lenmode': 'fraction'},
                         'colorscale': [[0.0, 'rgb(158,1,66)'], [0.1,
                                        'rgb(213,62,79)'], [0.2,
                                        'rgb(244,109,67)'], [0.3,
                                        'rgb(253,174,97)'], [0.4,
                                        'rgb(254,224,139)'], [0.5,
                                        'rgb(255,255,191)'], [0.6,
                              